# JAX Bellman Filter Debug

This notebook provides a minimal working example to debug the JAX-based Bellman filter implementation.

In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from functions.simulation import DFSV_params, simulate_DFSV
from functions.filters import DFSVBellmanFilter

# Set random seed for reproducibility
np.random.seed(42)

# Create a minimal test model
K = 1  # Number of factors
N = 2  # Number of observed series

# Define model parameters
lambda_r = np.array([[1.0], [0.5]])  # Simple factor loadings
Phi_f = np.array([[0.9]])            # Factor persistence
Phi_h = np.array([[0.95]])           # Volatility persistence
mu = np.array([-1.0])                # Log-volatility mean
sigma2 = np.ones(N) * 0.1            # Measurement noise
Q_h = np.array([[0.2]])              # Volatility of log-volatility

# Create parameter object
params = DFSV_params(
    N=N,
    K=K,
    lambda_r=lambda_r,
    Phi_f=Phi_f,
    Phi_h=Phi_h,
    mu=mu,
    sigma2=sigma2,
    Q_h=Q_h
)

print("Model parameters created successfully")

Model parameters created successfully


In [2]:
# Generate synthetic data
T = 20  # Just a few time points for testing
y, factors, log_vols = simulate_DFSV(params, T=T, seed=42)

print("Data generated:")
print(f"y shape: {y.shape}")
print(f"factors shape: {factors.shape}")
print(f"log_vols shape: {log_vols.shape}")

Data generated:
y shape: (20, 2)
factors shape: (20, 1)
log_vols shape: (20, 1)


In [3]:
# Initialize the filter
try:
    bf = DFSVBellmanFilter(params)
    print("\nBellman filter initialized")
    print("\nJAX parameters:")
    print(f"lambda_r shape: {bf.jax_lambda_r.shape}")
    print(f"sigma2 shape: {bf.jax_sigma2.shape}")
    print(f"mu shape: {bf.jax_mu.shape}")
    print(f"Phi_f shape: {bf.jax_Phi_f.shape}")
    print(f"Phi_h shape: {bf.jax_Phi_h.shape}")
    print(f"Q_h shape: {bf.jax_Q_h.shape}")
except Exception as e:
    print(f"Error during initialization: {e}")


Bellman filter initialized

JAX parameters:
lambda_r shape: (2, 1)
sigma2 shape: (2, 2)
mu shape: (1, 1)
Phi_f shape: (1, 1)
Phi_h shape: (1, 1)
Q_h shape: (1, 1)


In [4]:
# Initialize state and test prediction step
try:
    # Initialize state
    state0, cov0 = bf.initialize_state(y)
    print(f"Initial state shape: {state0.shape}")
    print(f"Initial covariance shape: {cov0.shape}")
    print(f"\nInitial state:\n{state0}")
    
    # Test prediction step
    predicted_state, predicted_cov = bf.predict(state0, cov0)
    print("\nPrediction step completed")
    print(f"Predicted state:\n{predicted_state}")
    
    # Verify covariance matrices
    try:
        np.linalg.cholesky(predicted_cov)
        print("\nPredicted covariance is positive definite ✓")
    except np.linalg.LinAlgError:
        print("\nWARNING: Predicted covariance is not positive definite!")
        print("Eigenvalues:", np.linalg.eigvals(predicted_cov))
except Exception as e:
    print(f"Error during prediction: {e}")

Initial state shape: (2, 1)
Initial covariance shape: (2, 2)

Initial state:
[[ 0.]
 [-1.]]

Prediction step completed
Predicted state:
[[ 0.]
 [-1.]]

Predicted covariance is positive definite ✓


In [5]:
import tensorboard

# Check if TensorBoard is available
try:
    # Use a local directory for TensorBoard logs
    logdir = "./jax-trace"
    
    # Start TensorBoard
    %load_ext tensorboard
    %tensorboard --logdir {logdir}
    print(f"TensorBoard started with logs in {logdir}. Navigate to the URL above to view profiling data.")
except ModuleNotFoundError:
    print("TensorBoard not found. Please install with: pip install tensorboard jupyter_tensorboard")
    print("Then restart the kernel and try again.")

Reusing TensorBoard on port 6006 (pid 81118), started 0:02:50 ago. (Use '!kill 81118' to kill it.)

TensorBoard started with logs in ./jax-trace. Navigate to the URL above to view profiling data.


In [ ]:
jax.profiler.start_trace(log_dir=logdir)
observation = y[0:1, :].T.reshape(-1, 1)
print(f"Observation shape: {observation.shape}")
print(f"Observation values:\n{observation}")

# Check JAX objective function
print("\nTesting JAX objective function...")
alpha_test = predicted_state.copy()
alpha_test[0]=0.7
# Convert inputs for JAX
jax_alpha = jnp.array(alpha_test)
jax_pred = jnp.array(predicted_state)
jax_I_pred = jnp.array(np.linalg.inv(predicted_cov))
jax_obs = jnp.array(observation)

# Test objective function with new parameter order (removed K, N params)
obj_val, grad_val = bf.obj_and_grad_fn(jax_alpha, jax_pred, jax_I_pred, jax_obs,
                            bf.jax_lambda_r, bf.jax_sigma2)
print(f"Objective value at predicted state: {float(obj_val)}")

# Test gradient with new parameter order
print(f"Gradient at predicted state:\n{np.array(grad_val)}")
# Perform update step
print("\nPerforming update step...")

updated_state, updated_cov, log_likelihood = bf.update(
    predicted_state, predicted_cov, observation
)


print("\nUpdate step completed")
print(f"Updated state:\n{updated_state}")
print(f"Log-likelihood: {log_likelihood}")

# Verify updated covariance
try:
    np.linalg.cholesky(updated_cov)
    print("\nUpdated covariance is positive definite ✓")
except np.linalg.LinAlgError:
    print("\nWARNING: Updated covariance is not positive definite!")
    print("Eigenvalues:", np.linalg.eigvals(updated_cov))
jnp.array(updated_state).block_until_ready()
jax.profiler.stop_trace()

2025-03-12 15:56:40.220005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741791400.371234   82523 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741791400.399981   82523 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1741791400.702404   82523 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741791400.702431   82523 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741791400.702433   82523 computation_placer.cc:177] computation placer alr

Observation shape: (2, 1)
Observation values:
[[0.]
 [0.]]

Testing JAX objective function...
Objective value at predicted state: 1.1323174503064932
Gradient at predicted state:
[ 2.44209542 -1.16597865]

Performing update step...
Update step took 2.856068 seconds

Update step completed
Updated state:
[[ 0.        ]
 [-0.99919711]]
Log-likelihood: -0.39687066649718206

Updated covariance is positive definite ✓


In [ ]:
with jax.profiler.trace("/tmp/jax-trace"):
    print("Running full filter...")
    filtered_states, filtered_covs, log_likelihood = bf.filter(y)
    print("\nFilter completed successfully!")
    print(f"Total log-likelihood: {log_likelihood}")
    print(f"\nFiltered states shape: {filtered_states.shape}")
    print(f"Filtered covs shape: {filtered_covs.shape}")
    
    # Compare with true states
    print("\nCorrelation with true states:")
    factor_corr = jnp.corrcoef(filtered_states[:, 0], factors[:, 0])[0, 1]
    vol_corr = jnp.corrcoef(filtered_states[:, 1], log_vols[:, 0])[0, 1]
    print(f"Factor correlation: {factor_corr:.4f}")
    print(f"Log-volatility correlation: {vol_corr:.4f}")
    vol_corr.block_until_ready()

Running full filter...


Bellman Filter Progress: 100%|██████████| 20/20 [00:46<00:00,  2.35s/it]